https://www.kaggle.com/code/karnikakapoor/lyrics-generator-rnn/notebook?select=Songs.csv

# Training Process

In [7]:
# import pandas as pd
# from datasets import Dataset
# 
# df = pd.read_csv("Songs.csv")
# df = df.dropna(subset=["Lyrics"])
# df = df[df["Lyrics"].str.len() > 100]  # keep meaningful lyrics
#   
# lyrics_ds = Dataset.from_pandas(df[["Lyrics"]].rename(columns={"Lyrics": "text"}))

In [3]:
# from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
# 
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# tokenizer.pad_token = tokenizer.eos_token
# 
# def tokenize_fn(example):
#     tokens = tokenizer(
#         example["text"],
#         padding="max_length",
#         truncation=True,
#         max_length=256,
#         return_tensors="pt"
#     )
#     tokens["labels"] = tokens["input_ids"].clone()
#     return tokens
# 
# tokenized_ds = lyrics_ds.map(tokenize_fn, batched=True)
# tokenized_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
# 
# # Load model
# model = GPT2LMHeadModel.from_pretrained("gpt2")
# model.resize_token_embeddings(len(tokenizer))
# 
# # Define training args
# training_args = TrainingArguments(
#     output_dir="./banger_model",
#     overwrite_output_dir=True,
#     num_train_epochs=3,
#     per_device_train_batch_size=2,
#     save_steps=100,
#     logging_steps=20,
#     save_total_limit=2,
#     prediction_loss_only=True
# )
# 
# # Train
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_ds,
#     tokenizer=tokenizer
# )
# 
# trainer.train()

Map:   0%|          | 0/744 [00:00<?, ? examples/s]

Step,Training Loss
20,3.151700
40,2.980700
60,2.829400
80,2.791500
100,2.876700
120,2.725900
140,2.858100
160,2.638400
180,2.832600
200,2.832800


TrainOutput(global_step=1116, training_loss=2.531502740784785, metrics={'train_runtime': 9962.3499, 'train_samples_per_second': 0.224, 'train_steps_per_second': 0.112, 'total_flos': 291601907712000.0, 'train_loss': 2.531502740784785, 'epoch': 3.0})

In [4]:
# trainer.save_model("./banger_gpt2_model")
# tokenizer.save_pretrained("./banger_gpt2_model")

('./banger_gpt2_model\\tokenizer_config.json',
 './banger_gpt2_model\\special_tokens_map.json',
 './banger_gpt2_model\\vocab.json',
 './banger_gpt2_model\\merges.txt',
 './banger_gpt2_model\\added_tokens.json')

# RUN THIS:

In [4]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
import re
from collections import Counter

def audited_generate(
    prompt,
    model,
    tokenizer,
    dataset_texts,
    similarity_threshold=0.85,
    max_length=150,
    ngram_size=5
):
    # === Setup ===
    embed_model = SentenceTransformer("all-MiniLM-L6-v2")

    def clean(text):
        return re.sub(r"[^a-zA-Z0-9\s]", "", text).lower()

    cleaned_dataset = [clean(txt) for txt in dataset_texts]
    dataset_embeddings = embed_model.encode(cleaned_dataset, convert_to_tensor=True)

    # === Generate text ===
    generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
    gen_output = generator(prompt, max_length=max_length, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)[0]['generated_text']
    cleaned_gen = clean(gen_output)
    gen_embedding = embed_model.encode(cleaned_gen, convert_to_tensor=True)

    # === Similarity check ===
    similarities = util.cos_sim(gen_embedding, dataset_embeddings)[0]
    top_score = float(similarities.max())
    most_similar_idx = int(similarities.argmax())

    print(f"\nSimilarity to dataset: {top_score:.3f}")
    print(f"\nClosest known lyric (manual inspection):\n{dataset_texts[most_similar_idx]}...\n")

    if top_score >= similarity_threshold:
        print("Rejected: too similar to existing lyric (semantic similarity).")
        return None

    # === Line-level plagiarism check ===
    gen_lines = set([line.strip() for line in gen_output.lower().splitlines() if line.strip()])
    for line in dataset_texts[most_similar_idx].lower().splitlines():
        if line.strip() in gen_lines and line.strip() != "":
            print(f"Rejected: line-level match found — \"{line.strip()}\"")
            return None

    # === N-gram overlap check ===
    def get_ngrams(text, n):
        words = text.split()
        return Counter([" ".join(words[i:i+n]) for i in range(len(words)-n+1)])

    gen_ngrams = get_ngrams(cleaned_gen, ngram_size)
    song_ngrams = get_ngrams(cleaned_dataset[most_similar_idx], ngram_size)
    overlap = gen_ngrams & song_ngrams
    if sum(overlap.values()) > 0:
        print(f"Rejected: n-gram overlap detected ({sum(overlap.values())} matches)")
        print("Common phrase example:", list(overlap.keys())[0])
        return None

    # === Toxicity filter ===
    if any(word in cleaned_gen for word in ["kill", "rape", "bitch", "n***", "f***"]):
        print("Rejected: toxic language detected.")
        return None

    print("Passed audit.\n")
    return gen_output

In [2]:
import pandas as pd
from datasets import Dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def clean_lyrics(text):
    return re.sub(r"[^a-zA-Z0-9\s]", "", text).lower()

df = pd.read_csv("Songs.csv")
df = df.dropna(subset=["Lyrics"])
df = df[df["Lyrics"].str.len() > 100]  # keep meaningful lyrics
  
lyrics_ds = Dataset.from_pandas(df[["Lyrics"]].rename(columns={"Lyrics": "text"}))

dataset_texts = df["Lyrics"].astype(str).map(clean_lyrics).tolist()

model = GPT2LMHeadModel.from_pretrained("./banger_gpt2_model")
tokenizer = GPT2Tokenizer.from_pretrained("./banger_gpt2_model")

In [9]:
output = audited_generate("Under the moonlight she", model, tokenizer, dataset_texts)
if output:
    print(output)

C:\Users\nitza\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Similarity to dataset: 0.101

Passed audit.

Under the moonlight she painted my face
With your hand on my cheek
And I kissed her over the face of love
In exchange of an oath to you
Bam, I love you
You're my best friend tonight

Come in and ride home tonight
In this quiet, bright city
I'm riding on your shoulders
If you wanna share you should do the work for me
Let the moon out for the night
The sky is filled with sunshine
Oh, come


In [13]:
output = audited_generate("My love", model, tokenizer, dataset_texts)

if output:
    print(output)

C:\Users\nitza\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



Similarity to dataset: 0.546

Closest known lyric (manual inspection):
perhaps love is like a resting place a shelter from the storm
it exists to give you comfort it is there to keep you warm
and in those times of trouble when you are most alone
the memory of love will bring you home
perhaps love is like a window perhaps an open door
it invites you to come closer it wa...

✅ Passed audit.

My love, love, love, love
Like a knife through the heart of you
As I've held to you and I've broken you down

What do you expect from love, love, love?
So far I've been the one not to love you
And what do you expect from a lover?

(But don't take this at face value, can you?)
Well we'll still be with you for a little while
Maybe just in a little bit of


In [16]:
output = audited_generate("Watermelon", model, tokenizer, dataset_texts)

if output:
    print(output)

C:\Users\nitza\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



Similarity to dataset: 0.489

Closest known lyric (manual inspection):
stay for a little while
share a glass beside the fire
time goes by so fast so fast
lets make it last

stay stay and talk awhile
share a laugh share a smile
days go by so fast so fast
lets make it last

oh years they roll and fall away
all the colors slowly fade to grey

stay or watch another round
s...

✅ Passed audit.

Watermelon and a soft brown rose
Cinnamon rolled for the day
Cinnamon pie with caramelise frosting

The light's on the porch, summer's early
Beware the winter's coming
The lights, oh, are bright

I don't feel so bright anymore
The sun's out of the window, summer's hot

Come and stay with me, come

Come and stay with me, come

I'm so scared of the world now,


In [18]:
output = audited_generate("I just wanna party", model, tokenizer, dataset_texts)

if output:
    print(output)

C:\Users\nitza\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



Similarity to dataset: 0.565

Closest known lyric (manual inspection):
i am not trying to seduce you
would you like me to seduce you
is that what youre trying to tell me
yall ready

hey youre just too funky for me
i gotta get inside of you
and ill show you heaven
if you let me
hey youre just too funky for me
i gotta get inside i gotta get inside
i gotta get inside of you
so when will that be

i watch your fingers working overtime overtime
ive got to thinking that they should be mine oh
id love to see you naked baby
id like to think that sometime maybe tonight
if thats alright yeah

hey youre just too funky for me
i gotta get inside of you
wont let you go i wont let you no no
hey youre just too funky for me
i gotta get inside i gotta get inside
i gotta get inside of you
ill make you love me

i watch you drinking and i take my time take my time
i watch you sinking all of that cheap red wine oh
ive got to see you naked baby
id like to think that sometime maybe tonight
my goals in sight y

In [8]:
output = audited_generate("Hello", model, tokenizer, dataset_texts)

if output:
    print(output)

C:\Users\nitza\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Similarity to dataset: 0.528

Closest known lyric (manual inspection):
billie

what do you want from me why dont you run from me
what are you wondering what do you know
why arent you scared of me why do you care for me
when we all fall asleep where do we go

come here
say it spit it out what is it exactly
youre payin is the amount cleanin you out am i satisfactory
today im thinkin about the things that are deadly
the way im drinkin you down
like i wanna drown like i wanna end me

step on the glass staple your tongue ahh
bury a friend try to wake up ahahh
cannibal class killing the son ahh
bury a friend i wanna end me

i wanna end me
i wanna i wanna i wanna end me
i wanna i wanna i wanna

what do you want from me why dont you run from me
what are you wondering what do you know
why arent you scared of me why do you care for me
when we all fall asleep where do we go

listen
keep you in the dark what had you expected
me to make you my art and make you a star
and get you connected
ill meet